# 01_OPERA_phase_unwrap

Here, we introduce the concept of Interferometric Synthetic Aperture Radar (InSAR) and the difficulties in measuring accurate phase differences. In our case, we want these phase differences to act as an estimation for snow water equivalent

In [2]:
import coincident
import geopandas as gpd
import xarray as xr
from shapely.geometry import box
import numpy as np
import pandas as pd
import rasterio
from skimage.restoration import unwrap_phase
import matplotlib.pyplot as plt

### What is InSAR?

**Interferometric Synthetic Aperture Radar (InSAR)** is a remote sensing technique that measures surface displacement by analyzing the phase difference between two radar images of the same area. These images are collected at different times using radar sensors on satellites. The fundamental idea behind InSAR is that the phase difference between corresponding pixels in two SAR images contains information about the movement of the Earth's surface in the line-of-sight direction of the radar sensor.

![InSAR Diagram](../images/insar_diagram.jpg)

The process begins with the acquisition of two radar images, commonly referred to as a "master" and "slave" images (but let's refer to them as a "source" image and a "second" image), taken from essentially the same position in a satellite's orbit at different times. These images are compared to generate an **interferogram**, which captures the phase differences between the two radar signals. When the radar waves are reflected back to the sensor, they carry information about the distance between the satellite and the surface below. 

**Phase** refers to the position of a wave in its cycle. In the context of radar, it describes the relative position of the radar signal when it reaches the satellite's sensor. The **phase difference** between the signals from the two images measures the change in distance between the satellite and the surface - think of it as a measure of distance. When we talk about **wrapped phase**, we mean that the phase difference is confined to a specific range, usually between -π and π, due to the periodic nature of the radar signal. If the phase difference exceeds this range, it "wraps around" to stay within -π to π, causing the values to reset. This is called "wrapped" phase and how our data is stored:

$$
\varphi = W \left( \varphi_{\text{topography}} + \varphi_{\text{deformation}} + \varphi_{\text{atmosphere}} + \varphi_{\text{orbit}} + \varphi_{\text{noise}} \right)
$$

Where:
- $\varphi_{\text{topography}}$ represents the phase due to surface topography.
- $\varphi_{\text{deformation}}$ accounts for phase changes due to surface displacement.
- $\varphi_{\text{atmosphere}}$ includes phase shifts caused by atmospheric effects.
- $\varphi_{\text{orbit}}$ represents errors from satellite orbit inaccuracies.
- $\varphi_{\text{noise}}$ includes phase shifts from random noise.
- $W$ is the wrap operator that bounds the total phase between -π and π.

In our case, we're only concerned about $\varphi_{\text{topography}}$ and $\varphi_{\text{deformation}}$ due to the limited scope of coursework final projects. $\varphi_{\text{topography}}$ and $\varphi_{\text{atmosphere}}$ have the biggest influence on discerning $\varphi_{\text{deformation}}$ in terms of error, so we'll focus on $\varphi_{\text{topography}}$ given the 'complexity' of fitting an atmospheric model.

In **InSAR**, the phase difference directly corresponds to **surface displacement**. This displacement, particularly in the **line-of-sight** direction of the sensor, is encoded in the wrapped phase. By **unwrapping** the phase, we can obtain continuous measurements of surface motion over a region, which is essential for monitoring phenomena such as **earthquakes**, **land subsidence**, and **volcanic activity** (and in our case, snow water equivalent). Unwrapping involves converting the wrapped phase (with values between -π and π) into a continuous map that reflects the true displacement, but more on that later in this notebook.

> [!NOTE]  
> Coregistration, the process of aligning two radar images so that they match perfectly in space, is a crucial step in InSAR. However, we are using NASA's OPERA CSLC (Coregistered SLC) products, which have already undergone this process, so we will skip this step. For more information on coregistration and its importance, check out this https://www.gamma-rs.ch/uploads/media/2002-4_PhaseUnwrapping.pdf.

In [4]:
# load in our wrapped phase data from 00_SAR_intro.ipynb
fn_1 = "/home/jehayes/sh_final/cslc/asc/OPERA_L2_CSLC-S1_T049-103322-IW2_20180103T010924Z_20240428T014600Z_S1B_VV_v1.1.h5"
fn_2 = "/home/jehayes/sh_final/cslc/asc/OPERA_L2_CSLC-S1_T049-103322-IW2_20180115T010923Z_20240428T051456Z_S1B_VV_v1.1.h5"

dsR = xr.open_dataset(fn_1,
                      group='data',
                      engine='h5netcdf')['VV'].rename(dict(x_coordinates='x', y_coordinates='y'))
tR = pd.to_datetime(xr.open_dataset(fn_1, 
                                    group='identification')['zero_doppler_start_time'].data.astype('U')) # Unicode string format
dsS = xr.open_dataset(fn_2,
                      group='data',
                      engine='h5netcdf')['VV'].rename(dict(x_coordinates='x', y_coordinates='y'))

tS = pd.to_datetime(xr.open_dataset(fn_2, group='identification')['zero_doppler_start_time'].data.astype('U'))

ifg = dsR * np.conj(dsS)
ifg.attrs['reference'] = tR
ifg.attrs['secondary'] = tS
da = xr.apply_ufunc(np.angle, ifg)
ds = da.to_dataset(name='phase')
ds.rio.write_crs('EPSG:32613', inplace=True)

<xarray.Dataset> Size: 409MB
Dimensions:      (x: 20670, y: 4941)
Coordinates:
  * x            (x) float64 165kB 7.14e+05 7.14e+05 ... 8.173e+05 8.173e+05
  * y            (y) float64 40kB 4.218e+06 4.218e+06 ... 4.169e+06 4.169e+06
    spatial_ref  int64 8B 0
Data variables:
    phase        (y, x) float32 409MB nan nan nan nan nan ... nan nan nan nan

##### Topographic Phase and Its Removal

When generating an InSAR interferogram, one of the key contributors to the observed phase difference is the **topography** of the surface being observed. $\varphi_{\text{topography}}$ arises due to the variation in the elevation of the surface, which causes different radar signal paths to have different lengths. In simple terms, higher elevations increase the distance between the radar and the surface, while lower elevations reduce it. These differences create phase shifts that are purely due to topographical features, not due to any deformation or displacement of the surface.

The topographic phase is a common artifact in InSAR measurements because it distorts the true surface displacement signal. Without removing it, we would not be able to distinguish between actual displacement (such as due to land subsidence or earthquakes) and phase shifts that are simply caused by changes in surface elevation.

To isolate the true deformation signal, we need to **subtract the topographic phase**. This is typically done by using a reference Digital Elevation Model (DEM). The DEM provides a detailed map of the surface elevation and allows us to calculate the phase contribution due to topography at each point in the radar image. Once this topographic phase is modeled, it can be subtracted from the interferogram, leaving us with a phase signal that better represents the surface displacement (plus any residual errors, such as atmospheric phase shifts or noise).

The process works as follows:

1. **Generate the topographic phase**: Using the reference DEM, calculate the expected phase shift at each pixel based on the elevation.
$$
\varphi_{\text{topography}} = \left( \frac{\lambda}{4 \pi} \right) \times \left( \frac{h}{R} \right)
$$
- **$\lambda$**: Wavelength of the radar signal (in meters).
- **$4\pi$**: Constant factor used in the formula for calculating the topographic phase.
- **$h$**: Elevation of the surface from the Digital Elevation Model (DEM) (in meters).
- **$R$**: Radar line-of-sight distance from the sensor to the surface (in meters).
2. **Subtract the topographic phase**: Remove the calculated topographic phase from the interferogram, which leaves a phase signal primarily due to displacement.

In our case, our reference DEM is the Copernicus 30 GLO-DEM at 30m spatial resolution due to ease of access.

This is done before unwrapping our phase since $\varphi_{\text{topography}}$ is itself a phase term expressed in radians and thus behaves the same way as the wrapped phase.

In [ ]:
# create search polygon from bounds of SAR images so we can pull in our cop30 data
polygon = box(*ds.rio.bounds())
gs_search = gpd.GeoSeries([polygon], crs="EPSG:32612")
gs_search_wgs84 = gs_search.to_crs("EPSG:4326")
gf_search_fn = "/home/jehayes/sh_final/SWEet-InSAR/data/search_bbox.geojson"
gs_search_wgs84.to_file(gf_search_fn,
                        driver="GeoJSON")

In [ ]:
crs_utm = 'EPSG:32613'
cop30_dem_fn = '/home/jehayes/sh_final/SWEet-InSAR/data/asc_cop30.tif'
! /home/jehayes/finesst_24/fetch_dem/download_global_DEM.py -demtype COP30_E -vector_fn $gf_search_fn -out_proj $crs_utm -out_fn $cop30_dem_fn

In [ ]:
with rasterio.open(cop30_dem_fn) as cop30_src:
    arr_cop30 = cop30_src.read(1)  # Read the DEM data as a NumPy array
    cop30_transform = cop30_src.transform  # Affine transform for georeferencing

In [ ]:
# Calculate the topographic phase
# The formula for topographic phase is: phase_topo = (4 * pi / wavelength) * (h / R)
# where h is the elevation from the DEM and R is the radar line-of-sight distance
# For simplicity, assume R is constant or use a simplified model
R = 1  # Simplified radar line-of-sight distance (arbitrary value for simplicity)
# Calculate the topographic phase (assuming R = 1)
wavelength = 0.056  # Sentinel-1 wavelength in m
topographic_phase = (4 * np.pi / wavelength) * (arr_cop30 / R)

##### The phase difference between two corresponding pixels is measured modulo 2π, resulting in a wrapped phase that ranges from -π to π. This wrapping occurs because the phase is inherently periodic; a phase difference of 2π corresponds to a full cycle, and any phase difference beyond this is indistinguishable from a smaller value within the -π to π range. Storing phase within this range simplifies the measurement process and ensures that phase differences are consistently represented.

To actually get an interpretable deformation measurement, we neep to 'unwrap' the phase to obtain a continuous phase map. This involves integrating the phase differences between neighboring pixels. The unwrapped phase φ_unwrapped at a pixel (i, j) is given by:

$$
φ_{\text{unwrapped}}(i, j) = φ_{\text{wrapped}}(i, j) + 2π \sum_{k=1}^{i} \sum_{l=1}^{j} Δφ(k, l)
$$
https://hyp3-docs.asf.alaska.edu/guides/insar_product_guide

where Δφ(k, l) is the phase difference between neighboring pixels. Various algorithms, such as the Minimum Cost Flow method, are employed to perform phase unwrapping, aiming to minimize the total phase difference across the entire image.
https://www.gamma-rs.ch/uploads/media/2002-4_PhaseUnwrapping.pdf

This process is challenging, though, and many factors can affect the quality of the unwrapped phase map. For instance, noise, phase discontinuities (when phase differences are very large, the algorithm might mistakenly treat it as a jump of 2π), and path dependence (the sequence of pixels or data points traversed during the unwrapping process - the summation over neighboring pixels (k, l) can produce different results if the order or direction of integration changes) can all impact the accuracy of the unwrapping process.
https://igppweb.ucsd.edu/~fialko/insar/phase_unwrap.pdf
https://arxiv.org/abs/1806.01832

In [ ]:
wrapped_phase = ds['phase'].data  # Extract as a NumPy array
wavelength = 0.056  # Sentinel-1 wavelength in meters
# https://scikit-image.org/docs/stable/auto_examples/filters/plot_phase_unwrap.html
# https://github.com/scikit-image/scikit-image/blob/main/skimage/restoration/unwrap.py
unwrapped_phase = unwrap_phase(wrapped_phase)
# NOTE: the above took 90+ minutes and still didnt finish. I'm really trying to find the best
# way to do this with OPERA CLSCs but am stuck for now. Ideally, I would use hyp3-isce2 but
# I'm still learning that software